# Backpropagation Algorithm

In [14]:
import pandas as pd
import json
import pprint as pp
import math
import copy
import numpy as np
from IPython.display import display 

In [15]:
def weighted_sum(values, weights):
    res = 0
    for value, weight in zip(values, weights):
        res += value * weight
    return res

In [16]:
def actual_output(weighted_sum): 
    return 1/(1 + math.exp(-weighted_sum))

In [17]:
def delta_ou(target_output, actual_output_ou):
    return (target_output - actual_output_ou) * actual_output_ou * (1-actual_output_ou)

In [18]:
def new_weight_to_ou(weight_to_ou, learning_coef, delta_ou, actual_output_hu):
    return weight_to_ou + learning_coef * delta_ou * actual_output_hu

In [19]:
def delta_hu(delta_ou, weight_to_ou, actual_output_hu):
        return delta_ou * weight_to_ou * actual_output_hu * (1 - actual_output_hu) 

In [20]:
def new_weight_to_hu(weight_to_hu, learning_coef, delta_hu, value_iu):
    return weight_to_hu + learning_coef * delta_hu * value_iu

In [21]:
def terminate_training(actual_outputs_of_ou, target_outputs, lower_margin, upper_margin):
    for i, target_output in enumerate(target_outputs):
        for j, item in enumerate(target_output):
            if item == 1:
                if actual_outputs_of_ou[i][j] <= upper_margin:
                    return False
            else:
                if actual_outputs_of_ou[i][j] >= lower_margin:
                    return False
    # print(True, actual_outputs_of_ou)
    return True

In [22]:
def avg(items):
    res = [([0]*len(items[0][0])).copy() for _ in range(len(items[0]))]
    # print(res)
    for j in range(len(items[0][0])):
        for i in range(len(items[0])):
            for item in items:
                res[i][j] += item[i][j]
            # print(res, i, j)
            res[i][j] /= len(items)
    return res

In [23]:
def bpa1(dataset, num_epochs = 1, roundup = 4):
    log = []

    epoch = 0
    final_weights_to_hus = copy.deepcopy(dataset['weights_to_hidden_units'])
    final_weights_to_ous = copy.deepcopy(dataset['weights_to_output_units'])
    target_outputs = copy.deepcopy(dataset['target_outputs_list'])

    while epoch <= num_epochs:
        actual_outputs_of_hus = []
        actual_outputs_of_ous = []
        list_weights_to_hus = []
        list_weights_to_ous = []
        
        for i, inputs in enumerate(dataset['inputs_list']):
            weights_to_hus = copy.deepcopy(final_weights_to_hus)
            weights_to_ous = copy.deepcopy(final_weights_to_ous)
            # print(weights_to_hus)

            actual_output_of_hus = []
            for weights_to_hu in weights_to_hus:
                actual_output_of_hus.append(actual_output(weighted_sum(inputs, weights_to_hu)))
            
            actual_output_of_ous = []
            for weights_to_ou in weights_to_ous:
                actual_output_of_ous.append(actual_output(weighted_sum(actual_output_of_hus, weights_to_ou)))

            temp_weights_to_ous = copy.deepcopy(weights_to_ous)
            delta_ous = []
            for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                delta_ous.append(delta_ou(target_outputs[i][k], actual_output_of_ou))
                for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                    weights_to_ous[k][j] = new_weight_to_ou(weights_to_ous[k][j], dataset['learning_coef'], delta_ous[k], actual_output_of_hu)

            # print(f'delta_ous: {delta_ous}')
            # print(f'weights_to_ous: {weights_to_ous}')

            delta_hus = []
            for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                temp_delta_hu = 0
                for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                    temp_delta_hu += delta_hu(delta_ous[k], temp_weights_to_ous[k][j], actual_output_of_hu)
                delta_hus.append(temp_delta_hu)

            for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                for i, input in enumerate(inputs):
                    weights_to_hus[j][i] = new_weight_to_hu(weights_to_hus[j][i], dataset['learning_coef'], delta_hus[j], input)
            
            # print(f'delta_hus: {delta_hus}')
            # print(f'weights_to_hus: {weights_to_hus}')

            actual_outputs_of_ous.append(actual_output_of_ous)
            actual_outputs_of_hus.append(actual_output_of_hus)
            list_weights_to_ous.append(weights_to_ous)
            list_weights_to_hus.append(weights_to_hus)
        # print(len(list_weights_to_hus))
        # pp.pprint(list_weights_to_hus)
        final_weights_to_hus = avg(list_weights_to_hus)
        # print(final_weights_to_hus)
        final_weights_to_ous = avg(list_weights_to_ous)

        sub_log = {}
        sub_log.update({"1.epoch": epoch})
        # sub_log.update({"2.inputs": np.round(inputs, roundup)})
        # sub_log.update({"2.target_outputs": target_outputs.copy()})
        # sub_log.update({"2.actual_outputs_of_hus": np.round(actual_outputs_of_hus, roundup)})
        sub_log.update({"2.actual_outputs": actual_outputs_of_ous.copy()})
        sub_log.update({"3.weights_to_output_units": weights_to_ous.copy()})
        sub_log.update({"4.weights_to_hidden_units": np.round(weights_to_hus, roundup)})
        log.append(sub_log)
        # log=sub_log

        if terminate_training(actual_outputs_of_ous, target_outputs, dataset['lower_margin'], dataset['upper_margin']): break
        epoch += 1
    return log

In [24]:
with open('dataset.json', 'r') as file:
    dataset = json.load(file)

In [27]:
for data in dataset:
    pp.pprint(data)
    log = bpa1(data, 100000, 5)
    # df = pd.DataFrame(log[:10])
    df = pd.DataFrame(log[-1:])
    # df = pd.DataFrame([log])
    display(df.style)
    break

{'inputs_list': [[0, 0, 0],
                 [1, 0, 0],
                 [0, 1, 0],
                 [1, 1, 0],
                 [0, 0, 1],
                 [1, 0, 1],
                 [0, 1, 1],
                 [1, 1, 1]],
 'learning_coef': 0.5,
 'lower_margin': 0.1,
 'target_outputs_list': [[0], [1], [1], [0], [0], [1], [1], [0]],
 'upper_margin': 0.9,
 'weights_to_hidden_units': [[0.2, 0.3, 0.4], [0.5, 0.6, 0.7]],
 'weights_to_output_units': [[0.8, 0.9]]}


,1.epoch,2.actual_outputs,3.weights_to_output_units,4.weights_to_hidden_units
0,88018,"[[0.03597712033496381], [0.9208084732043368], [0.9208083428349533], [0.07915731488431352], [0.0026389183188500038], [0.9483307549108131], [0.9483308869312006], [0.09999986436893588]]","[[-32.932245231223405, 26.34738004576651]]",[[ 0.98913 0.98913 -0.05225] [ 8.0856 8.08651 -0.49839]]
